In [35]:
import pandas as pd


# 1. Exploratory Data Analysis

In [46]:
#read in csv file data
#read train rows from 
trainCSV = pd.read_csv('train.csv',nrows=820000)
testCSV = pd.read_csv('test.csv')

rows = len(trainCSV)
rows2 = len(testCSV)

#check proper number of rows from the csv files
print("# of rows from trainCSV " + str(rows))
print("# of rows from testCSV " + str(rows2))

# of rows from trainCSV 820000
# of rows from testCSV 359


In [48]:
#check associated # of sentiment values from train csv
numNeg = ((trainCSV['Sentiment'] == 0).sum())
print("# of negatives from trainCSV " + str(numNeg))

numPos = ((trainCSV['Sentiment'] == 1).sum())
print("# of positives from trainCSV " + str(numPos))

#check if there are any improper sentiment values in traincsv
numNA = ((trainCSV['Sentiment'] >= 2).sum())
print("# of neither positive or negative from trainCSV " + str(numNA))

# of negatives from trainCSV 800000
# of positives from trainCSV 20000
# of neither positive or negative from trainCSV 0


In [51]:
#check if there are null values in the csv files
print("Null values in trainCSV")
print(trainCSV.isnull().sum())

print("Null values in testCSV")
print(testCSV.isnull().sum())

Null values in trainCSV
Index        0
Sentiment    0
Text         0
dtype: int64
Null values in testCSV
Index        0
Sentiment    0
Text         0
dtype: int64


# 2: Text Preprocessing.

In [60]:
#change all text to lowercase
trainCSV['Text'] = trainCSV['Text'].str.lower();
testCSV['Text'] = trainCSV['Text'].str.lower();

#check to make sure operation worked and changed to lowercase
print(trainCSV)

         Index  Sentiment                                               Text
0            0          0  @switchfoot http://twitpic.com/2y1zl - awww, t...
1            1          0  is upset that he can't update his facebook by ...
2            2          0  @kenichan i dived many times for the ball. man...
3            3          0    my whole body feels itchy and like its on fire 
4            4          0  @nationwideclass no, it's not behaving at all....
...        ...        ...                                                ...
819995  819995          1  we just met some awesome people at t.g.i fridays 
819996  819996          1  oh my goodness . my feet are the most sensitiv...
819997  819997          1                                  @hey_angy hahaha 
819998  819998          1  @lamartian30 pfft school is koo. i'm ranked 2n...
819999  819999          1  stuffed. working out, shower,movie night, thn ...

[820000 rows x 3 columns]


In [61]:
#check to make sure operation worked and changed to lowercase
print(testCSV)

     Index  Sentiment                                               Text
0        0          1  @switchfoot http://twitpic.com/2y1zl - awww, t...
1        1          1  is upset that he can't update his facebook by ...
2        2          1  @kenichan i dived many times for the ball. man...
3        3          1    my whole body feels itchy and like its on fire 
4        4          1  @nationwideclass no, it's not behaving at all....
..     ...        ...                                                ...
354    492          1  stupid movies we watched... mirrors ugggggh......
355    494          0      http://twitpic.com/2y2yi - i love you, buck. 
356    495          1                 last one!!! but still not done!!! 
357    496          0                                      homework.... 
358    497          0  i don't understand why things get taken away. ...

[359 rows x 3 columns]


In [65]:
#remove special characters
trainCSV['Text'] = trainCSV['Text'].str.replace('[^a-zA-Z0-9 \n\.]', '')
testCSV['Text'] = testCSV['Text'].str.replace('[^a-zA-Z0-9 \n\.]', '')

#remove numerical numbers
trainCSV['Text'] = trainCSV['Text'].str.replace('\d', '')
testCSV['Text'] = testCSV['Text'].str.replace('\d', '')

#print for visual check that special characters and digits removed
print(trainCSV)

C:\Users\Wil\AppData\Local\Temp\ipykernel_15340\576637622.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  trainCSV['Text'] = trainCSV['Text'].str.replace('[^a-zA-Z0-9 \n\.]', '')
C:\Users\Wil\AppData\Local\Temp\ipykernel_15340\576637622.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  testCSV['Text'] = testCSV['Text'].str.replace('[^a-zA-Z0-9 \n\.]', '')
C:\Users\Wil\AppData\Local\Temp\ipykernel_15340\576637622.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  trainCSV['Text'] = trainCSV['Text'].str.replace('\d', '')


         Index  Sentiment                                               Text
0            0          0  switchfoot httptwitpic.comyzl  awww thats a bu...
1            1          0  is upset that he cant update his facebook by t...
2            2          0  kenichan i dived many times for the ball. mana...
3            3          0    my whole body feels itchy and like its on fire 
4            4          0  nationwideclass no its not behaving at all. im...
...        ...        ...                                                ...
819995  819995          1  we just met some awesome people at t.g.i fridays 
819996  819996          1  oh my goodness . my feet are the most sensitiv...
819997  819997          1                                    heyangy hahaha 
819998  819998          1  lamartian pfft school is koo. im ranked nd in ...
819999  819999          1  stuffed. working out showermovie night thn sle...

[820000 rows x 3 columns]


C:\Users\Wil\AppData\Local\Temp\ipykernel_15340\576637622.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  testCSV['Text'] = testCSV['Text'].str.replace('\d', '')


In [67]:
#print for visual check that special characters and digits removed
print(testCSV)

     Index  Sentiment                                               Text
0        0          1  switchfoot httptwitpic.comyzl  awww thats a bu...
1        1          1  is upset that he cant update his facebook by t...
2        2          1  kenichan i dived many times for the ball. mana...
3        3          1    my whole body feels itchy and like its on fire 
4        4          1  nationwideclass no its not behaving at all. im...
..     ...        ...                                                ...
354    492          1  stupid movies we watched... mirrors ugggggh......
355    494          0              httptwitpic.comyyi  i love you buck. 
356    495          1                       last one but still not done 
357    496          0                                      homework.... 
358    497          0  i dont understand why things get taken away. i...

[359 rows x 3 columns]


# 3: Linguistic Feature Extraction.

In [70]:
#NLP using bag of words, token each word


# 4: Build your sentiment classification model.

# 5: Model evaluation.